In [5]:
import numpy as np

from ase import Atoms
from ase.io import read, write

from pyace import create_multispecies_basis_config
from pyace.activelearning import compute_B_projections

In [16]:
basis_config = {
  "deltaSplineBins": 0.001,
  "elements": ['C'],

  "embeddings": {
    "ALL": {
      "npot": 'FinnisSinclairShiftedScaled', # ?
      "fs_parameters": [ 1, 1],
      "ndensity": 1, # ?
    },
  },

  "bonds": {
    "ALL": {
      "radbase": "SBessel",
      "radparameters": [ 5.25 ],
      "rcut": 6, # ?
      "dcut": 0.01, # ?
    }
  },

  "functions": {
    # "number_of_functions_per_element": 1000,
    "ALL": {
        "nradmax_by_orders": [ 8,8,6],
        "lmax_by_orders"   : [ 0,6,4] }
  }
}


In [17]:
basis = create_multispecies_basis_config(basis_config)
basis

BBasisConfiguration(deltaSplineBins=0.001, funcspecs_blocks=['C', ])

In [18]:
path = "/home/evere/dev/quests/examples/gap20/Graphite.xyz"
atoms_list = read(path, index=":")
descriptor_ace_dict = compute_B_projections(basis, atoms_list)
descriptor_ace_dict

# # /p/lustre2/samanta1/Cu/step10_PotFit/test1/list20.txt
# with open(path) as f1:
#     for l1 in f1.readlines():
        # cfg1 = read(l1.split()[1] + '/OUTCAR')
#         dscrptr_ace = compute_B_projections(basis, atomic_env_list=[cfg1])
#         ace_list += list(dscrptr_ace[0])

{0: array([[ 5.30699625,  4.87772161,  4.63383655, ...,  0.06639799,
          0.03726733, -0.00743433],
        [ 5.30169047,  4.8585322 ,  4.57873826, ...,  4.82295864,
          0.13828475, -5.12933646],
        [ 5.30288263,  4.85515545,  4.57593924, ...,  4.78332148,
          0.13571487, -5.0994087 ],
        ...,
        [ 4.20097867,  1.80477498,  1.07852852, ..., -0.08517027,
         -0.71969204,  1.91552557],
        [ 4.20097867,  1.80477498,  1.07852852, ..., -0.08517027,
         -0.71969204,  1.91552557],
        [ 4.20097867,  1.80477498,  1.07852852, ..., -0.08517027,
         -0.71969204,  1.91552557]])}

In [20]:
descriptor_ace = descriptor_ace_dict[0]
descriptor_ace.shape

(12220, 1994)

In [21]:
from quests.descriptor import get_descriptors
from quests.entropy import perfect_entropy, diversity

h = 0.015
batch_size = 10000
H = perfect_entropy(descriptor_ace, h=h, batch_size=batch_size)
D = diversity(descriptor_ace, h=h, batch_size=batch_size)
print(H)
print(D)

8.468691558908088
9.184509909335528


In [70]:
np.savetxt('graphite_ace.txt', descriptor_ace, delimiter=' ')